In [1]:
import traceback

import pandas as pd
import tushare as ts
import time

type_max = {'北交所': 29.8, '创业板': 19.8}
# 连续涨停数
grow_up_count = 3

ts.set_token('your token here')
# pro = ts.pro_api()
tu_token = 'fba0f69c02f5e4399b8dd63ae895ca69b7a55122a545c6f0f2c2bef3'
pro = ts.pro_api(tu_token)
start_date = '20220101'


def get_single_grow(ts_code, name, market):
    """
    计算从一个非涨停开始，连续两个涨停之后第三天的涨跌幅
    :param ts_code:
    :return:
    """
    df = pro.daily(ts_code=ts_code, start_date=start_date)
    df_day = pd.concat([df.ts_code, df.trade_date, df.close, df.pct_chg], axis=1)
    df_day.columns = ['ts_code', 'trade_date', 'close', 'grow']
    market_max = type_max.get(market)
    if None is market_max:
        market_max = 9.8
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': [], '连续涨停数':[]}
    df_res = pd.DataFrame(list_industry)
    tmp_start = start_date
    # 涨停数
    count = 0
    for index, row in df_day.iterrows():
        if row['grow'] < market_max:
            break
        else:
            count += 1

        if count >= grow_up_count:
            tmp_start = row['trade_date']
            for k in list_industry.keys():
                list_industry[k].clear()
            list_industry['股票代码'].append(row['ts_code'])
            list_industry['股票名称'].append(name)
            list_industry['开始'].append(tmp_start)
            list_industry['结果'].append(row['grow'])
            list_industry['市场'].append(market)
            list_industry['连续涨停数'].append(count)
    df_res = pd.DataFrame(list_industry)
    if len(df_res) > 0:
        print(df_res)
    # print("%s : %s : %d" % (ts_code, name, df_res.shape[0]))
    return df_res


def get_single_data():
    df_all = pro.query('stock_basic', exchange='', list_status='L', fields='ts_code,name,market')
    list_industry = {'股票代码': [], '股票名称': [], '开始': [], '结果': [], '市场': [], '连续涨停数': []}
    df_res = pd.DataFrame(list_industry)
    size = df_all.shape[0]
    count = 1
    for index, row in df_all.iterrows():
        print("%d : %s , %s  %d / %d" % (grow_up_count, row['ts_code'], row['name'], index, size))
        try:
           df_tmp = get_single_grow(row['ts_code'], row['name'], row['market'])
           df_res = df_res.append(df_tmp)
        except Exception as e:
           traceback.format_exc()
        if df_res.shape[0] > 1:
            count += 1
        # if count > 3:
        #     break
    tag = f'推荐-{str(time.strftime("%Y-%m-%d"))}'
    df_res.to_csv('./res/%s.csv' % tag)
    print("*" * 50)
    print(tag)
    str_res = "总共出现： %d 次\n涨 %d 次， 跌 %d 次， 平 %d 次\n平均值： %f" % (df_res.shape[0],
                                                               df_res.loc[df_res["结果"] > 0].shape[0],
                                                               df_res.loc[df_res["结果"] < 0].shape[0],
                                                               df_res.loc[df_res["结果"] == 0].shape[0],
                                                               df_res['结果'].sum() / df_res.shape[0])
    print(str_res)
    file_path = "./res/%s.txt" % tag
    with open(file_path, "w+") as f:
        f.write(str_res)


get_single_data()

# df_res = get_single_grow("000856.SZ","aa","b")
# df_res

3 : 000001.SZ , 平安银行  0 / 4728
3 : 000002.SZ , 万科A  1 / 4728
3 : 000004.SZ , 国华网安  2 / 4728
3 : 000005.SZ , ST星源  3 / 4728
3 : 000006.SZ , 深振业A  4 / 4728
3 : 000007.SZ , *ST全新  5 / 4728
3 : 000008.SZ , 神州高铁  6 / 4728
3 : 000009.SZ , 中国宝安  7 / 4728
3 : 000010.SZ , 美丽生态  8 / 4728
3 : 000011.SZ , 深物业A  9 / 4728
3 : 000012.SZ , 南玻A  10 / 4728
3 : 000014.SZ , 沙河股份  11 / 4728
3 : 000016.SZ , 深康佳A  12 / 4728
3 : 000017.SZ , 深中华A  13 / 4728
3 : 000019.SZ , 深粮控股  14 / 4728
3 : 000020.SZ , 深华发A  15 / 4728
3 : 000021.SZ , 深科技  16 / 4728
3 : 000023.SZ , 深天地A  17 / 4728
3 : 000025.SZ , 特力A  18 / 4728
3 : 000026.SZ , 飞亚达  19 / 4728
3 : 000027.SZ , 深圳能源  20 / 4728
3 : 000028.SZ , 国药一致  21 / 4728
3 : 000029.SZ , 深深房A  22 / 4728
3 : 000030.SZ , 富奥股份  23 / 4728
3 : 000031.SZ , 大悦城  24 / 4728
3 : 000032.SZ , 深桑达A  25 / 4728
3 : 000034.SZ , 神州数码  26 / 4728
3 : 000035.SZ , 中国天楹  27 / 4728
3 : 000036.SZ , 华联控股  28 / 4728
3 : 000037.SZ , 深南电A  29 / 4728
3 : 000038.SZ , 深大通  30 / 4728
3 : 000039.SZ , 中集集团  31